#### Setting up Wikipedia as tool

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=wiki_wrapper)

wiki.name

'wikipedia'

#### Creating a Retrieval tool for custom web page

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents,OllamaEmbeddings())

retriever = vectordb.as_retriever()

retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\gaura\AppData\Local\Temp\ipykernel_25152\299460050.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(documents,OllamaEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029B7F110880>, search_kwargs={})

In [3]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
                                       "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

retriever_tool.name

'langsmith_search'

#### Setting up ARXIV as a tool

In [4]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name

'arxiv'

In [5]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Anaconda\\envs\\llm_venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029B3DD148B0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vecto

#### Setting up the llm model - llama 2

In [6]:

'''
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
llm
'''

from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama2")
llm



C:\Users\gaura\AppData\Local\Temp\ipykernel_25152\1034954243.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama2")


ChatOllama()

#### Importing prompt template from langchain hub

In [7]:
from langchain import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

#### Creating Agent

In [8]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [9]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) 

agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

#### Invoking Agent for responding to user input

In [14]:
#input = input("Please enter your enquiry - ")
agent_executor.invoke({"input":"What is Langchain"})



> Entering new AgentExecutor chain...
Langchain is a decentralized language model that allows users to create, share, and monetize their content in a web3 environment. It is built on top of the Ethereum blockchain and utilizes smart contracts to enable a decentralized and transparent ecosystem for content creators and consumers.

Langchain's main features include:

1. Decentralized Content Creation: Langchain allows users to create, publish, and share their content in the form of language models, without relying on any centralized platform or intermediary.
2. Token-Based Rewards: Langchain utilizes a token-based reward system, where creators are rewarded with cryptocurrency for creating and sharing valuable content.
3. Decentralized Governance: Langchain has a decentralized governance system, which allows users to have a say in the direction of the platform through voting and other mechanisms.
4. Interoperability: Langchain is built to be interoperable with other web3 platforms and p

{'input': 'What is Langchain',
 'output': "Langchain is a decentralized language model that allows users to create, share, and monetize their content in a web3 environment. It is built on top of the Ethereum blockchain and utilizes smart contracts to enable a decentralized and transparent ecosystem for content creators and consumers.\n\nLangchain's main features include:\n\n1. Decentralized Content Creation: Langchain allows users to create, publish, and share their content in the form of language models, without relying on any centralized platform or intermediary.\n2. Token-Based Rewards: Langchain utilizes a token-based reward system, where creators are rewarded with cryptocurrency for creating and sharing valuable content.\n3. Decentralized Governance: Langchain has a decentralized governance system, which allows users to have a say in the direction of the platform through voting and other mechanisms.\n4. Interoperability: Langchain is built to be interoperable with other web3 platf